In [1]:
import pandas as pd
import numpy as np
import os, sys, re

In [2]:
sys.path.append("./static/python/")

In [3]:
import userinput

In [4]:
reload(userinput)

<module 'userinput' from './static/python/userinput.pyc'>

In [5]:
fn = r"/Users/dblyon/CloudStation/CPR/BTW_GO/UserInput.txt"
fn = r"/Users/dblyon/modules/cpr/agotool/static/data/exampledata/exampledata_yeast.txt"
fn = r"/Users/dblyon/modules/cpr/agotool/static/data/exampledata/refactorUserinput.txt"
df = pd.read_csv(fn, sep='\t', decimal=",")#, converters={"background_int": float, "background_an": str, "foreground_an": str})

In [6]:
df.head(10)

,background_int,background_an,foreground_an
0,1.704742,P00549,P00549
1,2.669038,P00330,P00330
2,3.642069,P05743,P05743
3,4.640611,P02407-1;P14127-2,P02407
4,5.636197,P0C2H6-999;P0C2H7,P0C2H7
5,6.623177,P35997;P38711-4948,P38711
6,7.623177,P35997;P38711,P35997;P38711
7,NaN,ABC123,YOYOYO


In [7]:
df.loc[0, "background_int"]

1.70474233

In [8]:
# fh = open(fn, "r").read()
ui = userinput.Userinput(user_input_fn=fn, foreground_string=None, background_string=None,
col_foreground_an='foreground_an', col_background_an='background_an', col_background_int='background_int',
num_bins=100, decimal='.', method="abundance_correction")

In [9]:
ui.background_df

,background_an,background_int
0,P00549,1.704742
1,P00330,2.669038
2,P05743,3.642069
3,P02407;P14127,4.640611
4,P0C2H6;P0C2H7,5.636197
5,P35997;P38711,6.623177
7,ABC123,-1.000000


In [10]:
df.loc[0, "background_int"]

1.70474233

In [11]:
# np.histogram(df["background_int"], bins=10)
np.histogram(df.loc[pd.notnull(df["background_int"]), "background_int"], bins=10)

(array([1, 1, 0, 1, 1, 0, 1, 0, 1, 1]),
 array([ 1.70474233,  2.29658579,  2.88842924,  3.4802727 ,  4.07211616,
         4.66395961,  5.25580307,  5.84764653,  6.43948999,  7.03133344,
         7.6231769 ]))

In [12]:
ui.background_df

,background_an,background_int
0,P00549,1.704742
1,P00330,2.669038
2,P05743,3.642069
3,P02407;P14127,4.640611
4,P0C2H6;P0C2H7,5.636197
5,P35997;P38711,6.623177
7,ABC123,-1.000000


In [13]:
# import math

In [14]:
# import matplotlib.pyplot as plt
# %matplotlib inline

In [15]:
# arr = list(np.arange(10))
# arr += [2.3]*8
# hist, bin_edges = np.histogram(arr, bins=10, density=False)
# plt.hist(arr, bins=bin_edges)  # plt.hist passes it's arguments to np.histogram
# plt.show()

In [37]:
# good stuff

In [25]:
# foreground proteinGroups with intensities
intensity_foreground = []
for proteinGroup in ui.foreground_ser:
    an_first_in_proteinGroup = proteinGroup.split(";")[0]
    intensity_foreground.append(ui.an_2_intensity_dict[an_first_in_proteinGroup])
#     try:      
#         intensity_foreground.append(ui.an_2_intensity_dict[an_first_in_proteinGroup])
#     except KeyError:
#         # can't occur, since defaultdict
#         raise StopIteration

In [35]:
ui.foreground_df = pd.DataFrame(data=intensity_foreground, columns=["Int"])
ui.foreground_df["AN"] = ui.foreground_ser

In [39]:
df = ui.foreground_df
df

,Int,AN
0,1.704742,P00549
1,2.669038,P00330
2,3.642069,P05743
3,4.640611,P02407
4,5.636197,P0C2H7
5,6.623177,P38711
6,6.623177,P35997;P38711
7,-1.000000,YOYOYO


In [44]:
1.5*30

45.0

In [ ]:
# create bins
# which proteinGroups from background are within bin

In [40]:
num_bins = 100
bins = np.linspace(df["Int"].min(), df["Int"].max(), num_bins)
groups = df.groupby(pd.cut(df["Int"], bins))

In [45]:
# for group in groups:
#     print group
#     print